In [ ]:
# example code for graphing
import numpy as np
import matplotlib.pyplot as plt

#  Fairly fast for many datapoints, less fast for many dimensions
def find_pareto(data):
    is_Pareto = np.ones(data.shape[0], dtype = bool)
    for i, c in enumerate(data):
        # Keep any point with a lower cost
        if is_Pareto[i]:
            # This is where you would change for miniminzation versus maximization 

            # Minimization
            is_Pareto[is_Pareto] = np.any(data[is_Pareto]<c, axis=1)  

            # Maximization
            #is_Pareto[is_Pareto] = np.any(data[is_Pareto]>c, axis=1)  

            # And keep self
            is_Pareto[i] = True  

    # Downsample from boolean array
    Pareto_data = data[is_Pareto, :]

    # Sort data
    Pareto_out =  Pareto_data[np.argsort(Pareto_data[:,0])]

    #return is_Pareto
    return Pareto_out

# Create random list of values
N=20
myData = np.random.random((N,2))

# Include the trival Pareto points, i.e. always on or off
myData = np.vstack(([[0,1],[1,0]], myData))

# Use above routine to find pareto points
myPareto=find_pareto(myData)

# Calculate the Area under the Curve as a Riemann sum
auc = np.sum(np.diff(myPareto[:,0])*myPareto[0:-1,1])

# Create figure
plt.figure()

# Make sure font sizes are large enough to read in the presentation
plt.rcParams.update({'font.size': 14})

# Plot all points
plt.scatter(myData[:,0],myData[:,1],)

# Plot Pareto steps. note 'post' for minimization 'pre' for maximization
plt.step(myPareto[:,0], myPareto[:,1], where='post')
#plt.step(myPareto[:,0], myPareto[:,1], where='pre')

# Make sure you include labels
# Minimization
plt.title('Example of a Minimization Result\n with AUC = ' + str(auc))
plt.xlabel('False Negative')
plt.ylabel('False Positive')

# Maximization
#plt.title('Example of a Maximization Result')
#plt.xlabel('True Negative')
#plt.ylabel('True Positive')

plt.show()


